<a href="https://colab.research.google.com/github/Patternic10/CSCI-4170-Project-in-AI-ML/blob/main/Homework6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1328a5d56d703f94b99de730741b3429d533d74411e25b27ad489f1632fd5360
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's depe

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import numpy as np
from datasets import load_dataset

# 1. Use a smaller subset of data
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:10%]")  # Only 10% of data
train_test = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset, test_dataset = train_test['train'], train_test['test']



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# 2. Use the base model for faster training
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# 3. Load metrics once (more efficient)
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

# 4. Optimized preprocessing
def preprocess_function(examples):
    inputs = [doc[:2000] for doc in examples["article"]]  # Truncate early to save time
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer([h[:500] for h in examples["highlights"]], max_length=64, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Process data in larger batches
tokenized_train = train_dataset.map(preprocess_function, batched=True, batch_size=16)
tokenized_test = test_dataset.map(preprocess_function, batched=True, batch_size=16)

# 5. Combined metrics computation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute both metrics in parallel
    rouge_results = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    bleu_results = bleu.compute(
        predictions=decoded_preds,
        references=[[label] for label in decoded_labels]
    )

    return {
        "rouge1": round(rouge_results["rouge1"], 4),
        "rouge2": round(rouge_results["rouge2"], 4),
        "rougeL": round(rouge_results["rougeL"], 4),
        "bleu": round(bleu_results["bleu"], 2),
    }

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Map:   0%|          | 0/25839 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2872 [00:00<?, ? examples/s]

In [ ]:
# 6. Optimized training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./bart_results",
    evaluation_strategy="steps",
    eval_steps=250,  # Evaluate more frequently but with smaller batches
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=1,
    fp16=True,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("Starting training...")
trainer.train()

# Final evaluation
print("\nFinal Evaluation:")
results = trainer.evaluate(tokenized_test)
print(f"ROUGE-1: {results['eval_rouge1']:.4f}")
print(f"ROUGE-2: {results['eval_rouge2']:.4f}")
print(f"ROUGE-L: {results['eval_rougeL']:.4f}")
print(f"BLEU: {results['eval_bleu']:.2f}")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-acc77b6a1898>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Starting training...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: qazimjalil (qazimjalil-rensselaer-polytechnic-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Bleu
250,2.447900,1.852361,0.254900,0.108200,0.204900,0.020000
500,2.029600,1.798819,0.252400,0.109700,0.203200,0.020000
750,1.979900,1.779046,0.248100,0.106300,0.200300,0.020000
1000,1.949200,1.734048,0.250500,0.108800,0.202600,0.020000
1250,1.925000,1.721173,0.246700,0.106700,0.200300,0.020000
1500,1.912700,1.714484,0.251600,0.109500,0.203400,0.020000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



Final Evaluation:


ROUGE-1: 0.2520
ROUGE-2: 0.1100
ROUGE-L: 0.2035
BLEU: 0.02


# Task 2-- RL
A real-world application that can be formulated as a Markov Decision Process (MDP) is autonomous driving. In this context, an autonomous vehicle must navigate roads, avoid obstacles, and reach a destination efficiently while adhering to traffic rules. Let’s break this down into the components of an MDP: state space, action space, transition model, and rewards.

**State Space**

The state space represents all the information the autonomous vehicle needs to make decisions at any given moment. This could include:
The vehicle’s current position (e.g., GPS coordinates or lane position).
The vehicle’s speed and direction.
The positions, speeds, and types of nearby objects (e.g., other vehicles, pedestrians, traffic lights).
Road conditions (e.g., wet, dry, or icy).
Traffic signals and signs (e.g., red light, stop sign).
The destination or goal location.
The state is a combination of these factors, providing a snapshot of the vehicle’s environment and its own status.

**Action Space**

The action space consists of the possible decisions the vehicle can take at any point. These actions might include:
Accelerate (increase speed).
Decelerate (slow down or brake).
Turn left or right (steering).
Change lanes (left or right).
Stop completely.
Maintain current speed and direction.
The specific actions available might depend on the current state (e.g., turning isn’t possible if there’s no intersection).

**Transition Model**

The transition model describes how the state changes based on the vehicle’s actions and the environment. It’s probabilistic because the real world is unpredictable. For example:
If the vehicle accelerates, it’s likely to increase its speed and move closer to its destination, but there’s a chance another car cuts in front, forcing it to slow down.
If it changes lanes, it usually succeeds, but there’s a small chance of a collision if a nearby vehicle behaves unexpectedly.
If it stops at a red light, the next state depends on when the light turns green and how other drivers react.
The transition model captures these dynamics qualitatively: actions generally move the vehicle toward an expected outcome, but external factors (other drivers, weather, etc.) introduce uncertainty.

**Rewards**

The reward structure guides the vehicle toward safe, efficient, and legal behavior. It could be designed as follows:
Positive rewards for progress toward the destination (e.g., moving closer to the goal).
Small positive rewards for maintaining a safe speed or following traffic rules.
Negative rewards (penalties) for unsafe actions, like getting too close to another car or running a red light.
Large negative rewards for collisions, traffic violations, or delays that make the trip inefficient.
A big positive reward for safely arriving at the destination.
The goal is to balance efficiency (reaching the destination quickly) with safety and compliance, so the reward system incentivizes cautious but effective driving.


This problem fits the MDP framework because the vehicle’s decisions depend only on the current state (not the full history), the outcomes of actions are probabilistic, and the rewards provide a clear objective. Reinforcement learning could train the vehicle to optimize its policy—learning, for instance, when to slow down at a yellow light versus speeding through, based on past experiences and accumulated rewards. Autonomous driving is a great example of a complex, real-world challenge where MDPs can model decision-making under uncertainty.

In [2]:
import numpy as np
import random
from collections import defaultdict

class TicTacToeAgent:
    def __init__(self, alpha=0.5, gamma=0.9, epsilon=0.1):
        """
        Initialize the Q-learning agent with:
        - alpha: learning rate
        - gamma: discount factor
        - epsilon: exploration rate
        """
        self.q_values = defaultdict(lambda: np.zeros(9))  # Q-table: state -> action values
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.state_history = []

    def get_state_key(self, board):
        """Convert the board to an immutable tuple for hashing"""
        return tuple(board)

    def choose_action(self, board):
        """Choose action using ε-greedy policy"""
        legal_moves = [i for i, val in enumerate(board) if val == 0]

        if not legal_moves:
            return None

        if random.random() < self.epsilon:
            # Exploration: random move
            return random.choice(legal_moves)
        else:
            # Exploitation: best move from Q-values
            state_key = self.get_state_key(board)
            q_values = self.q_values[state_key]
            # Get the best action among legal moves
            return max(legal_moves, key=lambda a: q_values[a])

    def update_q_values(self, reward):
        """Update Q-values based on the game outcome"""
        for state, action in reversed(self.state_history):
            state_key = self.get_state_key(state)
            self.q_values[state_key][action] += self.alpha * (
                reward - self.q_values[state_key][action]
            )
            reward = self.gamma * reward  # Discount future rewards

    def record_move(self, state, action):
        """Record the state-action pair for later updates"""
        self.state_history.append((state.copy(), action))

    def reset_history(self):
        """Clear the move history for a new game"""
        self.state_history = []

class TicTacToeGame:
    def __init__(self):
        self.board = [0] * 9  # 0=empty, 1=X, 2=O
        self.current_player = 1  # X starts

    def reset(self):
        self.board = [0] * 9
        self.current_player = 1
        return self.board

    def make_move(self, position):
        """Make a move and return (new_board, done, winner)"""
        if self.board[position] != 0:
            return self.board, False, None  # Invalid move

        self.board[position] = self.current_player

        # Check for winner
        winner = self.check_winner()
        if winner is not None or self.is_board_full():
            return self.board, True, winner

        # Switch player
        self.current_player = 3 - self.current_player  # Switches between 1 and 2
        return self.board, False, None

    def check_winner(self):
        """Check if there's a winner, return player number or None"""
        winning_lines = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
            [0, 4, 8], [2, 4, 6]              # diagonals
        ]

        for line in winning_lines:
            a, b, c = line
            if self.board[a] == self.board[b] == self.board[c] != 0:
                return self.board[a]
        return None

    def is_board_full(self):
        return all(cell != 0 for cell in self.board)

    def print_board(self):
        symbols = {0: ' ', 1: 'X', 2: 'O'}
        for i in range(0, 9, 3):
            print(f"{symbols[self.board[i]]} | {symbols[self.board[i+1]]} | {symbols[self.board[i+2]]}")
            if i < 6:
                print("---------")

def train_agent(agent, episodes=10000):
    """Train the agent by playing against itself"""
    game = TicTacToeGame()

    for _ in range(episodes):
        state = game.reset()
        agent.reset_history()
        done = False
        current_player = 1

        while not done:
            if current_player == 1:
                action = agent.choose_action(state)
                if action is None:
                    break
                agent.record_move(state, action)
            else:
                # Random opponent during training
                legal_moves = [i for i, val in enumerate(state) if val == 0]
                action = random.choice(legal_moves) if legal_moves else None

            state, done, winner = game.make_move(action)
            current_player = 3 - current_player

        # Update Q-values based on game outcome
        if winner == 1:  # Agent won
            agent.update_q_values(1.0)
        elif winner == 2:  # Agent lost
            agent.update_q_values(-1.0)
        else:  # Draw
            agent.update_q_values(0.5)

def evaluate_agent(agent, num_games=100):
    """Evaluate the agent against a random opponent"""
    game = TicTacToeGame()
    results = {'win': 0, 'loss': 0, 'draw': 0}

    for _ in range(num_games):
        state = game.reset()
        done = False
        current_player = 1

        while not done:
            if current_player == 1:
                action = agent.choose_action(state)
            else:
                legal_moves = [i for i, val in enumerate(state) if val == 0]
                action = random.choice(legal_moves) if legal_moves else None

            if action is None:
                break

            state, done, winner = game.make_move(action)
            current_player = 3 - current_player

        if winner == 1:
            results['win'] += 1
        elif winner == 2:
            results['loss'] += 1
        else:
            results['draw'] += 1

    return results

# Train and evaluate the agent
agent = TicTacToeAgent(alpha=0.5, gamma=0.9, epsilon=0.1)
print("Training agent...")
train_agent(agent, episodes=10000)

print("\nEvaluation against random opponent:")
results = evaluate_agent(agent, num_games=1000)
print(f"Wins: {results['win']}, Losses: {results['loss']}, Draws: {results['draw']}")
print(f"Win rate: {results['win'] / 10}%")

# Interactive demo
def play_against_agent(agent):
    game = TicTacToeGame()
    state = game.reset()
    game.print_board()

    while True:
        # Human move
        try:
            move = int(input("Your move (0-8): "))
            if move < 0 or move > 8 or state[move] != 0:
                print("Invalid move!")
                continue
        except ValueError:
            print("Please enter a number 0-8")
            continue

        state, done, winner = game.make_move(move)
        game.print_board()

        if done:
            if winner == 2:
                print("You win!")
            else:
                print("It's a draw!")
            break

        # Agent move
        print("\nAgent's turn...")
        action = agent.choose_action(state)
        state, done, winner = game.make_move(action)
        game.print_board()

        if done:
            if winner == 1:
                print("Agent wins!")
            else:
                print("It's a draw!")
            break

print("\nPlay against the agent! (You are O, agent is X)")
play_against_agent(agent)

Training agent...

Evaluation against random opponent:
Wins: 917, Losses: 33, Draws: 50
Win rate: 91.7%

Play against the agent! (You are O, agent is X)
  |   |  
---------
  |   |  
---------
  |   |  
Your move (0-8): 0
X |   |  
---------
  |   |  
---------
  |   |  

Agent's turn...
X | O |  
---------
  |   |  
---------
  |   |  
Your move (0-8): 2
X | O | X
---------
  |   |  
---------
  |   |  

Agent's turn...
X | O | X
---------
O |   |  
---------
  |   |  
Your move (0-8): 8
X | O | X
---------
O |   |  
---------
  |   | X

Agent's turn...
X | O | X
---------
O | O |  
---------
  |   | X
Your move (0-8): 3
Invalid move!
Your move (0-8): 5
X | O | X
---------
O | O | X
---------
  |   | X
It's a draw!


In [3]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import random
from collections import defaultdict

class TicTacToeEnv(gym.Env):
    metadata = {'render_modes': ['human']}

    def __init__(self):
        self.board = np.zeros(9, dtype=int)
        self.current_player = 1
        self.done = False

        # Observation space: 9 cells with values 0 (empty), 1 (X), or 2 (O)
        self.observation_space = spaces.MultiDiscrete([3] * 9)

        # Action space: 9 possible moves (0-8)
        self.action_space = spaces.Discrete(9)

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.board = np.zeros(9, dtype=int)
        self.current_player = 1
        self.done = False
        return self.board.copy(), {}

    def step(self, action):
        if self.done:
            return self.board.copy(), 0, True, False, {}

        if self.board[action] != 0:
            # Invalid move
            return self.board.copy(), -1, True, False, {}

        # Apply move
        self.board[action] = self.current_player

        # Check winner
        winner = self._check_winner()
        if winner is not None:
            reward = 1 if winner == 1 else -1
            self.done = True
            return self.board.copy(), reward, self.done, False, {}

        if np.all(self.board != 0):
            # Draw
            self.done = True
            return self.board.copy(), 0.5, self.done, False, {}

        # Switch player
        self.current_player = 3 - self.current_player
        return self.board.copy(), 0, self.done, False, {}

    def _check_winner(self):
        winning_lines = [
            [0,1,2], [3,4,5], [6,7,8],  # Rows
            [0,3,6], [1,4,7], [2,5,8],  # Columns
            [0,4,8], [2,4,6]            # Diagonals
        ]
        for line in winning_lines:
            a, b, c = line
            if self.board[a] == self.board[b] == self.board[c] != 0:
                return self.board[a]
        return None

    def render(self):
        symbols = {0: ' ', 1: 'X', 2: 'O'}
        print("-------------")
        for i in range(0, 9, 3):
            print(f"| {symbols[self.board[i]]} | {symbols[self.board[i+1]]} | {symbols[self.board[i+2]]} |")
            print("-------------")

class QLearningAgent:
    def __init__(self, env, alpha=0.5, gamma=0.9, epsilon=0.1):
        self.env = env
        self.q_table = defaultdict(lambda: np.zeros(env.action_space.n))
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate

    def get_state_key(self, state):
        return tuple(state)

    def choose_action(self, state):
        if random.random() < self.epsilon:
            return self.env.action_space.sample()
        else:
            state_key = self.get_state_key(state)
            return np.argmax(self.q_table[state_key])

    def update_q_table(self, state, action, reward, next_state):
        state_key = self.get_state_key(state)
        next_state_key = self.get_state_key(next_state)

        current_q = self.q_table[state_key][action]
        max_future_q = np.max(self.q_table[next_state_key])
        new_q = (1 - self.alpha) * current_q + self.alpha * (reward + self.gamma * max_future_q)

        self.q_table[state_key][action] = new_q

def train_agent(env, agent, episodes=10000):
    for _ in range(episodes):
        state, _ = env.reset()
        total_reward = 0
        done = False

        while not done:
            action = agent.choose_action(state)
            next_state, reward, done, truncated, _ = env.step(action)
            agent.update_q_table(state, action, reward, next_state)
            state = next_state.copy()
            total_reward += reward

def evaluate_agent(env, agent, num_episodes=100):
    wins = 0
    draws = 0
    losses = 0

    for _ in range(num_episodes):
        state, _ = env.reset()
        done = False
        total_reward = 0

        while not done:
            action = agent.choose_action(state)
            next_state, reward, done, truncated, _ = env.step(action)
            state = next_state.copy()
            total_reward += reward

        if total_reward == 1:
            wins += 1
        elif total_reward == -1:
            losses += 1
        else:
            draws += 1

    print(f"Win rate: {wins/num_episodes:.2%}")
    print(f"Loss rate: {losses/num_episodes:.2%}")
    print(f"Draw rate: {draws/num_episodes:.2%}")

# Create environment and agent
env = TicTacToeEnv()
agent = QLearningAgent(env)

# Training
print("Training agent...")
train_agent(env, agent, episodes=10000)

# Evaluation
print("\nEvaluation results:")
evaluate_agent(env, agent, num_episodes=1000)

# Interactive play
def play_vs_agent(env, agent):
    state, _ = env.reset()
    env.render()

    while True:
        # Human move
        try:
            action = int(input("Enter your move (0-8): "))
            if env.board[action] != 0:
                print("Invalid move!")
                continue
        except ValueError:
            print("Please enter a number 0-8")
            continue

        state, reward, done, truncated, _ = env.step(action)
        env.render()

        if done:
            if reward == 1:
                print("You win!")
            elif reward == -1:
                print("Agent wins!")
            else:
                print("Draw!")
            break

        # Agent move
        print("\nAgent's move:")
        action = agent.choose_action(state)
        state, reward, done, truncated, _ = env.step(action)
        env.render()

        if done:
            if reward == 1:
                print("Agent wins!")
            elif reward == -1:
                print("You win!")
            else:
                print("Draw!")
            break

print("\nPlay against the agent (you are O, agent is X):")
play_vs_agent(env, agent)

Training agent...

Evaluation results:
Win rate: 79.90%
Loss rate: 17.30%
Draw rate: 2.80%

Play against the agent (you are O, agent is X):
-------------
|   |   |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------
Enter your move (0-8): 1
-------------
|   | X |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------

Agent's move:
-------------
| O | X |   |
-------------
|   |   |   |
-------------
|   |   |   |
-------------
Enter your move (0-8): 5
-------------
| O | X |   |
-------------
|   |   | X |
-------------
|   |   |   |
-------------

Agent's move:
-------------
| O | X | O |
-------------
|   |   | X |
-------------
|   |   |   |
-------------
Enter your move (0-8): 8
-------------
| O | X | O |
-------------
|   |   | X |
-------------
|   |   | X |
-------------

Agent's move:
-------------
| O | X | O |
-------------
|   |   | X |
-------------
|   |   | X |
-------------
You win!


In [6]:
import numpy as np
import random

class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3))  # 0: empty, 1: X, -1: O
        self.game_over = False
        self.winner = None

    def reset(self):
        """Reset the board for a new game."""
        self.board = np.zeros((3, 3))
        self.game_over = False
        self.winner = None

    def get_state(self):
        """Return current board state as a hashable tuple."""
        return tuple(self.board.flatten())

    def is_valid_move(self, action):
        """Check if the move is valid (cell is empty)."""
        row, col = divmod(action, 3)
        return self.board[row, col] == 0

    def make_move(self, action, player):
        """Place player's mark (1 or -1) at the action position."""
        row, col = divmod(action, 3)
        self.board[row, col] = player
        self.check_game_over()

    def check_game_over(self):
        """Check if the game has ended and set winner."""
        # Check rows, columns, diagonals for a win
        for i in range(3):
            if abs(sum(self.board[i, :])) == 3:
                self.game_over = True
                self.winner = int(self.board[i, 0])
                return
            if abs(sum(self.board[:, i])) == 3:
                self.game_over = True
                self.winner = int(self.board[0, i])
                return
        if abs(self.board[0, 0] + self.board[1, 1] + self.board[2, 2]) == 3:
            self.game_over = True
            self.winner = int(self.board[0, 0])
            return
        if abs(self.board[0, 2] + self.board[1, 1] + self.board[2, 0]) == 3:
            self.game_over = True
            self.winner = int(self.board[0, 2])
            return
        # Check for draw
        if not any(0 in row for row in self.board):
            self.game_over = True
            self.winner = None  # Draw

    def print_board(self):
        """Display the current board state."""
        symbols = {0: '.', 1: 'X', -1: 'O'}
        for row in self.board:
            print(' '.join(symbols[int(x)] for x in row))
        print()

class QLearningAgent:
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.q_table = {}  # Dictionary for Q-values: (state, action) -> value
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

    def get_q_value(self, state, action):
        """Retrieve Q-value, default to 0 if unseen."""
        return self.q_table.get((state, action), 0)

    def choose_action(self, game):
        """Select action using epsilon-greedy policy."""
        state = game.get_state()
        available_actions = [i for i in range(9) if game.is_valid_move(i)]
        if not available_actions:
            return None
        if random.random() < self.epsilon:  # Exploration
            return random.choice(available_actions)
        # Exploitation: choose action with highest Q-value
        q_values = [self.get_q_value(state, a) for a in available_actions]
        max_q = max(q_values)
        best_actions = [a for a, q in zip(available_actions, q_values) if q == max_q]
        return random.choice(best_actions)

    def update_q(self, state, action, reward, next_state, game):
        """Update Q-value based on the Q-learning rule."""
        if next_state is None or game.game_over:  # Terminal state
            target = reward
        else:
            available_actions = [i for i in range(9) if game.is_valid_move(i)]
            max_q_next = max([self.get_q_value(next_state, a) for a in available_actions]) if available_actions else 0
            target = reward + self.gamma * max_q_next
        current_q = self.get_q_value(state, action)
        self.q_table[(state, action)] = current_q + self.alpha * (target - current_q)

def play_game(agent, verbose=False):
    """Play one game, updating agent's Q-table if training."""
    game = TicTacToe()
    state = game.get_state()
    while not game.game_over:
        # Agent (X) moves
        action = agent.choose_action(game)
        game.make_move(action, 1)
        if verbose:
            print("Agent (X) move:")
            game.print_board()
        if game.game_over:
            reward = 1 if game.winner == 1 else 0  # Win or draw
            agent.update_q(state, action, reward, None, game)
            break
        # Random opponent (O) moves
        available = [i for i in range(9) if game.is_valid_move(i)]
        if available:
            opp_action = random.choice(available)
            game.make_move(opp_action, -1)
            if verbose:
                print("Opponent (O) move:")
                game.print_board()
        next_state = game.get_state() if not game.game_over else None
        reward = -1 if game.winner == -1 else 0 if game.winner is None else 0
        agent.update_q(state, action, reward, next_state, game)
        state = next_state
    return game.winner

# Training
agent = QLearningAgent(alpha=0.1, gamma=0.9, epsilon=0.1)
num_episodes = 10000
wins, losses, draws = 0, 0, 0

print("Training the agent...")
for episode in range(num_episodes):
    winner = play_game(agent)
    if winner == 1:
        wins += 1
    elif winner == -1:
        losses += 1
    else:
        draws += 1
    if (episode + 1) % 1000 == 0:
        print(f"After {episode + 1} games: Wins={wins}, Losses={losses}, Draws={draws}")

# Demonstration Runs
print("\nDemonstration Runs (epsilon=0):")
agent.epsilon = 0  # Pure exploitation
for i in range(3):
    print(f"\nGame {i + 1}:")
    winner = play_game(agent, verbose=True)
    result = "Agent (X) wins" if winner == 1 else "Opponent (O) wins" if winner == -1 else "Draw"
    print(f"Result: {result}")

# Evaluation Metric
win_rate = wins / num_episodes
print(f"\nEvaluation Metric - Win Rate: {win_rate:.2f}")

Training the agent...
After 1000 games: Wins=788, Losses=129, Draws=83
After 2000 games: Wins=1654, Losses=196, Draws=150
After 3000 games: Wins=2532, Losses=253, Draws=215
After 4000 games: Wins=3441, Losses=292, Draws=267
After 5000 games: Wins=4342, Losses=330, Draws=328
After 6000 games: Wins=5245, Losses=370, Draws=385
After 7000 games: Wins=6162, Losses=403, Draws=435
After 8000 games: Wins=7079, Losses=428, Draws=493
After 9000 games: Wins=7985, Losses=469, Draws=546
After 10000 games: Wins=8895, Losses=508, Draws=597

Demonstration Runs (epsilon=0):

Game 1:
Agent (X) move:
. . X
. . .
. . .

Opponent (O) move:
. . X
. . .
. O .

Agent (X) move:
. . X
. X .
. O .

Opponent (O) move:
. . X
O X .
. O .

Agent (X) move:
. . X
O X .
X O .

Result: Agent (X) wins

Game 2:
Agent (X) move:
. . X
. . .
. . .

Opponent (O) move:
. . X
. . O
. . .

Agent (X) move:
. . X
. X O
. . .

Opponent (O) move:
. . X
. X O
. O .

Agent (X) move:
. . X
. X O
. O X

Opponent (O) move:
. . X
O X O
. 